In [ ]:
!pip install langchain langchain_core langchain_groq langchain_community langgraph

In [ ]:
from typing import TypedDict, Dict
from langgraph.graph import StateGraph, END
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables.graph import MermaidDrawMethod
from IPython.display import display , Image

In [ ]:
class State(TypedDict):
  query: str
  category: str
  sentiment: str
  response: str

In [ ]:
from langchain_groq import ChatGroq

llm = ChatGroq(
    temperature=0,
    groq_api_key = "groq_api_key",
    model_name = "llama-3.3-70b-versatile"
)
result = llm.invoke("What is Life")
result.content